# 强化学习
## 学习意义
深度学习如图像识别和语音识别解决的是感知的问题，强化学习解决的是决策的问题。人工智能的终极目的是通过感知进行智能决策。所以，将近年发展起来的深度学习技术与强化学习算法结合而产生的深度强化学习算法是人类实现人工智能终极目的的一个很有前景的方法。  
## 分类
* 分类1：
    * 基于策略的：策略优化方法，其又分为进化算法和策略梯度方法。
    * 基于价值的：动态规划方法，其又分为策略迭代方法和价值迭代方法。
* 分类2：
    * 确定性策略：每一个状态输出一个固定动作
    * 随机性策略：每个状态按照概率选择，也有探索的功能
* 分类3：
    * 模型相关：转移概率已知
    * 模型无关：转移概率未知
* 分类4：
    * 表格型：状态和动作空间是离散且数目较少
    * 函数逼近：状态和动作都是连续或者数目较多

# 强化学习基本概念
 * s:状态，例如棋盘的状态、智能体当前位置    
 * a:动作，例如下一步的方向、下一步的速度   
 * r:奖励：每走一步的回馈奖励  
 * pi:策略：在某个状态下走某一个动作的集合  
 * v(s):在某一个策略下，我们用价值函数来衡量某一个状态好不好用，定义：$$v(s) = E_{\pi}[\sum_{k=0}^{\infty} \gamma^{k} R_{k}] $$
 这里期望表示的选取不同动作得到的奖励和之后奖励的平均值，比较麻烦，很多时候无法计算。  
 * q(s,a):来衡量某个状态下选取某个动作好坏的值，定义：$$q(s,a) = R_{s,a} + E_{\pi}[ \sum_{k=1}^{\infty} \gamma^{k} R_{k}] $$
 和v(s)相比选取了某个确定的动作，需要计算的少了很多，但是后面的v(s)依然是不确定的，还是很难计算。

### 贝尔曼等式
* v(s)的贝尔曼等式：$$v(s)= \sum_{a \in A} \pi(s,a) q(s,a)= \sum_{a \in A}\pi(s,a)(R_{s,a}+\gamma \sum_{s' \in S}T_{s,a}^{s'}v(s')) $$
从这里可以看出计算非常复杂，每一状态的计算都依赖于之后可能的所有状态，因此无法进行精确计算。
* q(s,a)的贝尔曼等式：$$q(s,a)=R_{s,a} + \gamma \sum_{s' \in S} T_{s,a}^{s'} v(s')=R_{s,a} + \gamma \sum_{s' \in S} T_{s,a}^{s'} \sum_{a' \in A} \pi(s',a') q(s',a')$$
这里将v(s')进一步展开，可以看出也是灰常灰常麻烦。

### 价值简化
**这里的简化是有条件的，必须是满足马尔科夫特性，即当前状态能够表示所需要的所有信息，下个状态只跟当前状态有关系，未来的状态只跟当前状态有关，和之前无关。**
* v(s)前面可以看到，我们评价一个策略好坏主要是通过v(s)和q(s,a)，但是他们得值很难计算或者无法计算，因此需要对这两个进行迭代求解。$$v(s) = E(r + \gamma v(s')) $$这里假设s采取某个动作只可能转移到一个状态，这样就可以用下一步的值来计算这一步v(s),不断迭代使其稳定。  
* 同理可以得到：$$q(s,a) = r + \gamma v(s') = r + E(\gamma q(s',a')) $$
这里期望是对下个状态所有的动作选取的，s'是s采取动作a之后达到的状态。

# 常见算法
强化学习算法通常可以分为两类，基于价值的和基于策略的。
* 基于价值的强化学习：求解一个最优的价值函数，这样我们每一个状态都可以选择最优的那个动作，这样就实现了目标。
* 基于策略的强化学习：目的是找到一个最优的策略，即$a = \pi_\theta(s)$，这里我们只要选择一个$\theta$使得累计收益最大，这样也实现了目标。

## 策略评价算法
### 蒙特卡洛算法（MC）
主要思想是：用当前策略探索一个完整的状态动作奖励序列，然后从后面的状态往前面的状态进行迭代计算，不断更新状态对应的价值，从而完成价值评估。计算步骤如下：
* 探索得到一个完成的状态动作奖励序列 $s_1,a_1,r_1,....,s_k,a_k,r_k \sim \pi$
* 从最后状态向前计算：
    * $g_{s}(t) = r_{t} + \gamma r_{t+1} + ... + \gamma ^{k-t} r_{k} = r_{t} + \gamma g_s(t+1)$
    * 计算v(s)平均值$$n(s) = n(s) + 1$$$$v(s) = \frac{v(s)*(n(s)-1)+g_s(t)}{n(s)} $$

蒙特卡罗算法能够有效地求解模型无关的策略评估，但也存在一些问题。有时我们面临的强化学习问题是持续不断的。比如没有停止指令时，飞行器控制要求不停地根据姿势风向等因素调整，持续保持平稳飞行，这个时候无法使用蒙特卡洛算法。

### 时差算法（TD）
蒙特卡洛是利用某一个序列的奖励来估计某个状态的价值，但是这样很多时候都无法计算，因此TD算法利用接下来n步的性质来进行计算。计算步骤如下：
* 按照系统策略进行探索，在某个状态s执行动作转移到s'
* 更新v(s),$v(s) = v(s) + \alpha (r + \gamma v(s') - v(s))$,其中r是获得的奖励，$\alpha$为学习率，$\gamma$为衰减因子，$r+\gamma v(s')$为时差目标(TD Target),$r+\gamma v(s') - v(s)$为时差误差TD Error。

上面的称为TD(0),而关心后续$\lambda$个步骤的称为TD($\lambda$).

## 策略学习
### 两个问题
* 上面价值衡量使用的都是v(s),但是使用v(s)会存在一些问题(暂且不说)，因此接下来使用状态动作对q(s,a)来进行衡量价值。
* 贪婪策略:有时候如果进行一个固定的策略可能会进入死胡同，因此给它加一些不确定性会得到更多的可能性。

### 蒙特卡洛控制(MC Control)
这个控制算法和上面讲到MC评价的思想基本一样，不过有了两点不同
* 这里使用q(s,a)来进行策略的衡量
* 这里在每一步选择的时候，选择某个状态下q(s,a)最大的那个动作，作为输出

### SARSA
State Action Reward State Action算法实际就是状态动作价值版本的时差学习方法，基本思想不加赘述。注意一点下个q(s,a)是随机选的。$$q(s,a) = q(s,a) + \alpha (r + \gamma q(s',a') - q(s,a))$$

### Q learning
Q learning算法框架和SARSA类似，唯一的不同就是更新q(s,a)的公式不同，$$q(s,a) = q(s,a) + \alpha \{r +  max_{a'}\{\gamma q(s',a')\} - q(s,a)\} $$
这里q(s,a)选取的是下一个状态不同动作的最大值。

## 价值函数逼近
前面所用的强化学习算法基本上都是状态空间和动作空间离散的，而且数目较少，但是实际上我们所遇到的很少会有这样简单情况。上面的强化学习方法基本上都是对状态行为值函数进行表格存储，根据遇到的情况不断更新表格。而状态很多维数很大的情况下我们无法进行存储，就需要利用函数逼近的方法进行表示，一旦表示出了价值函数，然后就可以使用策略迭代或者值迭代的方法进行强化学习。

价值函数逼近分为参数逼近和非参数逼近，参数逼近又分为线性参数逼近和非线性参数逼近。这里主要使用参数逼近来近似值函数。  
对于不同的强化学习算法(MC,TD,q学习)可以构建不同的目标函数，然后利用参数来更新。

## 参考
[强化学习大讲堂](https://zhuanlan.zhihu.com/sharerl)  
[强化学习专栏](http://www.algorithmdog.com/%E5%BC%BA%E5%8C%96%E5%AD%A6%E4%B9%A0-%E9%A9%AC%E5%B0%94%E7%A7%91%E5%A4%AB%E5%86%B3%E7%AD%96%E8%BF%87%E7%A8%8B)